In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import sys

# Import API key
sys.path.append('..')
from settings import config

# Configure gmaps
gmaps.configure(api_key=config.g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Hobart,AU,46.99,light rain,-42.88,147.33,St Ives Hobart Accommodation Tasmania
1,1,Snyder,US,55.40,clear sky,32.72,-100.92,Holiday Inn Express & Suites Snyder
2,2,Kaitangata,NZ,48.99,overcast clouds,-46.28,169.85,Kaitangata Motor Camp
3,4,Bluff,NZ,46.00,overcast clouds,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
4,10,Kjollefjord,NO,48.76,few clouds,70.95,27.35,Hotel Nordkyn


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>City Name</b></dt><dt>{City}</dt>
<dt><b>County Code</b></dt><dt>{Country}</dt>
<dt><b>Weather Description</b></dt><dt>{Current Description}</dt>
<dt><b>Max Temp</b></dt><dt>{Max Temp} F</dt>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
max_temp = vacation_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,weights=max_temp,max_intensity=300,dissipating=False, point_radius=4)
marker_layer =  gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 

vacation_start = vacation_df.loc[vacation_df['City']=='Hereford']
vacation_end = vacation_df.loc[vacation_df['City']=='Skegness']
vacation_stop1 = vacation_df.loc[vacation_df['City']=='Truro']
vacation_stop2 = vacation_df.loc[vacation_df['City']=='Margate'] 
vacation_stop3 = vacation_df.loc[vacation_df['City']=='Hastings']

In [13]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = tuple(vacation_start.to_numpy()[0][5:7])
end = tuple(vacation_end.to_numpy()[0][5:7])
stop1 = tuple(vacation_stop1.to_numpy()[0][5:7])
stop2 = tuple(vacation_stop2.to_numpy()[0][5:7])
stop3 = tuple(vacation_stop3.to_numpy()[0][5:7])

In [14]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING" depending how close the cities are.
fig = gmaps.figure()
uk_trip = gmaps.directions_layer(start,end,waypoints=[stop1,stop2, stop3],travel_mode='DRIVING')
fig.add_layer(uk_trip)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,465,Hereford,GB,53.01,overcast clouds,52.06,-2.71,The Green Dragon Hotel
1,348,Truro,GB,53.60,light rain,50.27,-5.05,Mannings Hotel
2,412,Margate,GB,54.00,overcast clouds,51.38,1.39,Premier Inn Margate hotel
3,469,Hastings,GB,54.00,scattered clouds,50.86,0.57,Zanzibar Hotel


In [17]:
#ode, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt><b>City Name</b></dt><dt>{City}</dt>
<dt><b>County Code</b></dt><dt>{Country}</dt>
<dt><b>Weather Description</b></dt><dt>{Current Description}</dt>
<dt><b>Max Temp</b></dt><dt>{Max Temp} F</dt>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
city_marker= gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(city_marker)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))